In [0]:
import io

def load_vectors(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    data = {}
    for line in fin:
        tokens = line.rstrip().split(' ')
        data[tokens[0]] = map(float, tokens[1:])
    return data

In [2]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.vec.gz

--2019-05-31 03:48:23--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.vec.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.20.22.166, 104.20.6.166, 2606:4700:10::6814:6a6, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.20.22.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1325960915 (1.2G) [binary/octet-stream]
Saving to: ‘cc.en.300.vec.gz’

cc.en.300.vec.gz    100%[===================>]   1.23G  35.5MB/s    in 37s     

2019-05-31 03:49:00 (34.4 MB/s) - ‘cc.en.300.vec.gz’ saved [1325960915/1325960915]



In [0]:
!gunzip -k cc.en.300.vec.gz

In [0]:
# data = load_vectors('cc.en.300.vec')

In [0]:
from gensim.models.keyedvectors import KeyedVectors
model = KeyedVectors.load_word2vec_format('cc.en.300.vec', binary=False)
model.save_word2vec_format('wiki.el.txt', binary=False)

In [0]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"]="3"

In [86]:
from google.colab import files
files.upload()

Saving data_df.csv to data_df (1).csv


{'data_df.csv': b',reason,token,noun_phrase,len\n0,My presence is needed in home due to some personel work.,"[\'presence\', \'needed\', \'home\', \'due\', \'personel\', \'work\']","[\'presence\', \'home\', \'personel\', \'work\']",4\n1,Out of valley,[\'valley\'],[\'valley\'],1\n2,back pain and rest,"[\'back\', \'pain\', \'rest\']","[\'pain\', \'rest\']",2\n3,Not feeling well,"[\'not\', \'feeling\', \'well\']","[\'not\', \'feeling\', \'well\']",3\n4,Not Feeling Well,"[\'not\', \'feeling\', \'well\']","[\'not\', \'feeling\', \'well\']",3\n5,Need work concentration,"[\'need\', \'work\', \'concentration\']","[\'need\', \'work\', \'concentration\']",3\n6,Personal stuff,"[\'personal\', \'stuff\']",[\'stuff\'],1\n7,Out of Valley,[\'valley\'],[\'valley\'],1\n8,"Out of KTM, Stuck in Banda","[\'ktm\', \'stuck\', \'banda\']",[\'strike\'],1\n9,Uncomfortable due to tonsillitis and neck pain,"[\'uncomfortable\', \'due\', \'tonsillitis\', \'neck\', \'pain\']","[\'tonsillitis\', \'neck\', \'pain\']",3

In [0]:
import numpy as np
import pandas as pd

files = pd.read_csv('data_df (1).csv')

In [88]:
files.head(20)

,Unnamed: 0,reason,token,noun_phrase,len
0,0,My presence is needed in home due to some pers...,"['presence', 'needed', 'home', 'due', 'persone...","['presence', 'home', 'personel', 'work']",4
1,1,Out of valley,['valley'],['valley'],1
2,2,back pain and rest,"['back', 'pain', 'rest']","['pain', 'rest']",2
3,3,Not feeling well,"['not', 'feeling', 'well']","['not', 'feeling', 'well']",3
4,4,Not Feeling Well,"['not', 'feeling', 'well']","['not', 'feeling', 'well']",3
5,5,Need work concentration,"['need', 'work', 'concentration']","['need', 'work', 'concentration']",3
6,6,Personal stuff,"['personal', 'stuff']",['stuff'],1
7,7,Out of Valley,['valley'],['valley'],1
8,8,"Out of KTM, Stuck in Banda","['ktm', 'stuck', 'banda']",['strike'],1
9,9,Uncomfortable due to tonsillitis and neck pain,"['uncomfortable', 'due', 'tonsillitis', 'neck'...","['tonsillitis', 'neck', 'pain']",3


In [0]:
import ast
def back_list(x):
  x_list = ast.literal_eval(x)
  return x_list
  
files['noun_phrase'] = files['noun_phrase'].apply(back_list)
  

In [90]:
type(files['noun_phrase'][0])

list

In [0]:
from pprint import pprint
def group_similar(x):
  
  if len(x)==0:
    return 'empty'
  
  list_group = ['unwell', 'home', 'strike', 'weather','travel', 'emergency', 'personal']
  similar = []
  
  for compare in list_group:  
    compare_with_all = []
    
    for token in x:
      try:
        similarity = model.similarity(compare,token)
        compare_with_all.append(similarity)
        
      except:
        compare_with_all.append(-1)
    
    similar.append(max(compare_with_all))
  
  if max(similar) > 0:
    max_index = similar.index(max(similar))
    return list_group[max_index]
  else:
    return 'other'

      

In [100]:
# group_similar(['unwell'])
files['classes'] = files['noun_phrase'].apply(group_similar)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [101]:
files[['reason','classes']].head(20)

,reason,classes
0,My presence is needed in home due to some pers...,home
1,Out of valley,home
2,back pain and rest,unwell
3,Not feeling well,unwell
4,Not Feeling Well,unwell
5,Need work concentration,home
6,Personal stuff,personal
7,Out of Valley,home
8,"Out of KTM, Stuck in Banda",strike
9,Uncomfortable due to tonsillitis and neck pain,unwell


In [102]:
files['classes'].value_counts()

home         4329
unwell       3473
travel       1093
emergency    1086
weather       648
personal      563
strike        431
other         235
empty           1
Name: classes, dtype: int64

In [104]:
files[files['classes']=='travel']

,Unnamed: 0,reason,token,noun_phrase,len,classes
38,38,Restriction on the movement of vehicle,"['restriction', 'movement', 'vehicle']","[restriction, movement, vehicle]",3,travel
41,41,Restriction in movement of vehicles.,"['restriction', 'movement', 'vehicle']","[restriction, movement, vehicle]",3,travel
43,43,Restriction in movement of vehicles.,"['restriction', 'movement', 'vehicle']","[restriction, movement, vehicle]",3,travel
54,54,Restriction in movement of vehicles.,"['restriction', 'movement', 'vehicle']","[restriction, movement, vehicle]",3,travel
57,57,Restriction on movement of vehicles,"['restriction', 'movement', 'vehicle']","[restriction, movement, vehicle]",3,travel
66,66,Restriction in the movement of vehicles,"['restriction', 'movement', 'vehicle']","[restriction, movement, vehicle]",3,travel
101,101,I have very important task to execute at about...,"['important', 'task', 'execute', 'oclock', 'ne...","[task, execute, oclock, time, loss, travel, of...",7,travel
109,109,Out of Kathmandu,['kathmandu'],[kathmandu],1,travel
112,112,Out of Kathmandu,['kathmandu'],[kathmandu],1,travel
118,118,Out of Kathmandu,['kathmandu'],[kathmandu],1,travel


In [0]:
files.to_csv('classes.csv')
from google.colab import files
files.download('classes.csv')